# Day 14

link: https://adventofcode.com/2022/day/14

ข้อนี้เก็บ input มาตรงๆ เป็น coordinate ของ rock segments

เราเตรียม method `fill_rock` ไว้ก่อนเลย เอาไว้เติมหินลง array โดยใช้เลข 1 แทนหิน
สังเกตว่าเรามี parameter `x_offset` อันนี้คือเอาไว้แปลงพิกัดแกน x เพราะตำแหน่ง array ของเราอาจไม่จำเป็นต้องเริ่มที่ 0

In [1]:
input = IO.foreach('in14.txt').to_a.map(&:strip)

segments = input.map{|line|
  line.split(' -> ').map{|coord| coord.split(',').map(&:to_i)}
}

def fill_rock(a, segments, x_offset)
  segments.each{|seg|
    seg.each_cons(2){|p1, p2|
      x1,y1 = p1
      x2,y2 = p2
      if x1 == x2
        y1,y2 = y2,y1 if y2 < y1
        y1.upto(y2){|y|
          a[y][x1 - x_offset] = 1
        }
      else
        x1,x2 = x2,x1 if x2 < x1
        x1.upto(x2){|x|
          a[y1][x - x_offset] = 1
        }
      end
    }
  }
end

y_max = segments.flat_map{|pt| pt.map{|x,y| y}}.max

nil

## Part 1

เราะ simulate ก้อนทรายตามโจทย์อย่างตรงไปตรงมา คือดูว่าตำแหน่งข้างล่างว่างไหม ถ้าไม่ไปดูล่างซ้าย ถ้ายังไม่ได้ไปล่างขวา ถ้าไม่ได้อีกก็หยุด

เนื่องจาก part นี้ไม่มีพื้น เราเลยรู้ว่าตำแหน่งซ้ายสุดที่ทรายจะไหลไปได้ ก็คือตำแหน่งซ้ายสุดที่มีหินอยู่ ทางขวาก็เช่นกัน

In [2]:
def fill(a, x, y)
  row = a[y+1]
  if row
    if row[x] == 0
      fill(a, x, y+1)
    elsif row[x-1] == 0
      fill(a, x-1, y+1)
    elsif row[x+1] == 0
      fill(a, x+1, y+1)
    else
      a[y][x] = 2
      true
    end
  else
    false
  end
end

def render(a)
  a.each{|x| puts x.map{|x| " █O"[x]}.join}
  nil
end

x_min = segments.flat_map{|pt| pt.map{|x,y| x}}.min
x_max = segments.flat_map{|pt| pt.map{|x,y| x}}.max

x_offset = x_min - 1
w = x_max - x_min + 3

a = 0.upto(y_max).map{ [0] * w }
fill_rock(a, segments, x_offset)

i = 0
i+=1 while fill(a, 500 - x_offset, 0)
puts i

render(a)

768
                                                                
                                                                
                                                                
                                                                
                                                                
                                                                
                                                                
                                                                
                                                                
                                         O                      
                                        OOO                     
                                       OOOOO                    
                                      OOOOOOO                   
                                     ██████OOO                  
                                    O     OOOOO                 
                     

                      ██████████████OOOO                        
                                   OOOOOO                       
                                  OOOOOOOO                      
                                 OOOOOOOOOO                     
                                OOOOOOOOOOO█                    
                               O████████████                    
                              OOO                               
                             OOOOO                              
                     █      OOOOOOO      █                      
                     █████████████████████  O                   
                                           OOO                  
                                          OOOOO                 
                                         ██████O                
                                        O     OOO               
                                       OOO   OOOOO              
                         

## Part 2

part หลังนี้ก็ยังทำเหมือนเดิม ต่างไปตรงที่ offset ต้องเผื่อมากขึ้น ซึ่งเรารู้ว่าตอนจบ ลิมิตความกว้างของฐานทรายคือสองเท่าของความสูง
ดังนั้นก็ต้องเผื่อ offset ไว้เท่าๆ กับความสูง

In [3]:
w = 2*(y_max + 2) + 1
x_offset = 500 - (y_max + 2)

a = 0.upto(y_max).map{ [0] * w }
fill_rock(a, segments, x_offset)
a << [0] * w
a << [1] * w

i = 0
while a[0][500-x_offset] == 0
  i += 1
  fill(a, 500 - x_offset, 0)  
end
puts i

26686


## Take 2

จุดอ่อนอย่างนึงของ code ข้างบน ก็คือเราทำงานซ้ำซ้อนค่อนข้างเยอะ เพราะ path ของก้อนทรายมันก็จะเป็นเส้นทางเดียวกับก้อนก่อนหน้าซะเป็นส่วนใหญ่

เราสามารถแก้ได้โดยเซฟ path เอาไว้ เมื่อขึ้นทรายก้อนใหม่ แทนที่จะไปเริ่มจาก (500, 0) ตั้งแต่ต้น เราก็ไปเริ่มจากตำแหน่งสุดท้ายใน trace แทน

In [4]:
def fill(a, x, y, trace)
  row = a[y+1]
  if row
    if row[x] == 0
      fill(a, x, y+1, trace << [x, y])
    elsif row[x-1] == 0
      fill(a, x-1, y+1, trace << [x, y])
    elsif row[x+1] == 0
      fill(a, x+1, y+1, trace << [x, y])
    else
      a[y][x] = 2
      trace
    end
  else
    nil
  end
end

# Part 1

x_min = segments.flat_map{|pt| pt.map{|x,y| x}}.min
x_max = segments.flat_map{|pt| pt.map{|x,y| x}}.max

x_offset = x_min - 1
w = x_max - x_min + 3

a = 0.upto(y_max).map{ [0] * w }
fill_rock(a, segments, x_offset)

i = 0
trace = fill(a, 500 - x_offset, 0, []) 
while trace
  i += 1
  x, y = trace.pop
  trace = fill(a, x, y, trace)
end
puts i

768


In [5]:
# Part 2

w = 2*(y_max + 2) + 1
x_offset = 500 - (y_max + 2)

a = 0.upto(y_max).map{ [0] * w }
fill_rock(a, segments, x_offset)
a << [0] * w
a << [1] * w

trace = fill(a, 500 - x_offset, 0, []) 
i = 1
until trace.empty?
  x, y = trace.pop
  trace = fill(a, x, y, trace)
  i += 1
end
puts i

26686
